<a href="https://colab.research.google.com/github/oliverwsd/finance/blob/master/set4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [0]:
'''
import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive/data"

os.chdir(path)#将当前目录指定到了云盘data文件夹
#os.listdir(path)
pd.read_csv('monthly_factors.csv')
'''

In [0]:
from io import StringIO
import csv


url = 'https://raw.githubusercontent.com/oliverwsd/finance/master/monthly_factors.csv'
factors = pd.read_csv(url)
factors.head()

In [0]:
# Read-in the factors for German market from the provided file 'monthly_factors.csv' (rf is risk free rate):
#   1. rm (Market factor)
#   2. SMB (Fama-French size factor)
#   3. HML (Fama-French value factor)
#   4. WML (Carhart momentum factor)
# Like above, convert the 'date' column to the datetime data type.
#
#factors         = pd.read_csv('monthly_factors.csv')
factors['date'] = pd.to_datetime(factors['date'], format ='%Y%m')
factors['rm_excess'] = factors['rm'] - factors['rf']
factors = factors[['date', 'rm_excess', 'rf']]  # We'll only use market excess returns and riskfree rate here
factors[['rm_excess', 'rf']] /= 100  # Bring numbers to actual values
factors.head()

In [73]:
# We will estimate an ARMA(1, 1) model on market excess returns and the riskfree rate:
#
#    r_t = \alpha + \beta * r_{t-1} + \epsilon_t + \theta * \epsilon_{t-1}   (1)
#
# This means: We need to come up with estimates for the parameters \alpha, \beta and \theta
#
# The problem is: \epsilon_{t-1} is not observed, we only have observations for r_t, r_{t-1}.
# We will follow a two-pass estimation to circumvent this problem. We first estimate the
# AR part of that model via OLS:
#
#    r_t = \alpha + \beta * r_{t-1} + \tilde{\epsilon}_t                      (2)
#
# Given this model, we can get a time series of \tilde{\epsilon}_t. With that, we can now
# estimate (1) via OLS, by just setting \epsilon_{t-1} = \tilde{\epsilon}_{t-1}.


# Task 1: Estimate AR model part (Step 1)
# ---------------------------------------

# Estimate the model (2) for market excess returns via OLS using the statsmodels package.
#
# Hint: If you are insecure what to do, write down the regression formulas for the factor
#       models of the last problem set and compare.
# Note: 'ar_model_rm_excess' should contain the OLS regression result object (after fitting)
#
# IMPORTANT: Use the constant as first variable in the regression. This also applied to the
#            next regression!
X = pd.DataFrame()
rm_excess_x = list(factors['rm_excess'])
rm_excess_y = list(factors['rm_excess'])
X['alpha'] = np.ones(len(factors['rm_excess'])-1)
X['rm_excess'] = rm_excess_x[0:-1]
Y = rm_excess_y[1:]
ar_model_rm_excess = sm.OLS(Y,X).fit()
print(ar_model_rm_excess.summary())

# Repeat the analysis above for the riskfree rate.
#
# Note: 'ar_model_rf' should contain the OLS regression result object
X = pd.DataFrame()
rf_x = list(factors['rf'])
rf_y = list(factors['rf'])
X['alpha'] = np.ones(len(factors['rf'])-1)
X['rf'] = rf_x[0:-1]
Y = rf_y[1:]
ar_model_rf = sm.OLS(Y,X).fit()
print(ar_model_rf.summary())


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     6.605
Date:                Thu, 05 Dec 2019   Prob (F-statistic):             0.0104
Time:                        01:16:11   Log-Likelihood:                 964.88
No. Observations:                 617   AIC:                            -1926.
Df Residuals:                     615   BIC:                            -1917.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
alpha          0.0038      0.002      1.833      0.0

In [83]:
# Task 2: Estimate full ARMA model (Step 2)
# -----------------------------------------
# tilde ???
tilde = rm_excess_y[1:] - (ar_model_rm_excess.params[0]+factors['rm_excess'][0:-1]*ar_model_rm_excess.params[1])
X = pd.DataFrame()
X['constant'] = np.ones(len(factors['rm_excess'])-1)   
X['rm_excess'] = rm_excess_x[0:-1]  
X['epsilon'] = tilde    
Y = rm_excess_y[1:]                                                             
arma_model_rm_excess = sm.OLS(Y,X).fit()
print(arma_model_rm_excess.summary())

#长度对的， 取值不对，范围需要修改

# Is the parameter for the AR part, beta, significant to the 5% confidence level in the ARMA model for
# market excess returns?
#
# If so, set ar_significant_rm_excess = 'yes'. Otherwise, set ar_significant_rm_excess = 'no'.
#ar_significant_rm_excess =

# Repeat the ARMA model estimation for the riskfree rate.
#
# Note: 'arma_model_rf' should contain the OLS regression result object (after fitting)
tilde = rf_y[1:] - (ar_model_rf.params[0]+factors['rf'][0:-1]*ar_model_rf.params[1])
X = pd.DataFrame()
X['constant'] = np.ones(len(factors['rf'])-2)   
X['rf'] = rf_x[0:-2]  
X['epsilon'] = tilde[0:-1]    
Y = rf_y[2:]                                                             
arma_model_rf = sm.OLS(Y,X).fit()
print(arma_model_rf.summary())
#arma_model_rf =
#print(arma_model_rf.summary())

# Is the parameter for the AR part, beta, significant to the 5% confidence level in the ARMA model for
# the riskfree rate?
#
# If so, set ar_significant_rf = 'yes'. Otherwise, set ar_significant_rf = 'no'.
#ar_significant_rf =


# Check of intermediate result (2):
#
# HINT: Check for yourself: arma_model_rm_excess.params[1] should be -0.072  (rounded to 3 digits)
#       Check for yourself: arma_model_rf.params[1] should be 0.988  (rounded to 3 digits)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 5.988e+33
Date:                Thu, 05 Dec 2019   Prob (F-statistic):               0.00
Time:                        01:33:28   Log-Likelihood:                 23188.
No. Observations:                 617   AIC:                        -4.637e+04
Df Residuals:                     614   BIC:                        -4.636e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
constant       0.0038   4.67e-19   8.04e+15      0.0